In [1]:
import pandas as pd
import numpy as np

In [2]:
targets = pd.read_csv("../../../../data/tp2/target_competencia_ids.csv")

In [3]:
targets["ref_hash"] = targets["ref_hash"].map(lambda x: x[:-3])

In [4]:
targets["ref_hash"] = targets["ref_hash"].astype("int64")

---

# Features Auctions

### Features auctions

In [5]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")


### Me quedo con datos del 24 en adelante

In [8]:
fecha_inicio = "2019-04-24"

In [9]:
fecha_fin = "2019-04-27"

In [10]:
auctions = auctions[auctions["date"] > fecha_inicio]

In [11]:
auctions.sort_values(by = "date",inplace = True)

In [12]:
targets_individuales = targets["ref_hash"].drop_duplicates().to_frame()\
                        .rename(columns = {"ref_hash":"device_id"})\
                        .set_index("device_id")

### Cantidad de apariciones discriminando períodos

In [10]:
# Creo columnas para luego separar
auctions['hora'] = auctions['date'].dt.hour
auctions['dia'] = auctions['date'].dt.day

---

#### Por hora

In [11]:
feature = auctions.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
feature = feature.unstack().fillna(0).astype(np.int64)
feature.columns = ["apariciones_en_hora_{}".format(hora) for hora in range(0,24)]

In [12]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

In [13]:
targets_individuales.isnull().sum().max()

686

##### Hay 686 dispositivos sin datos de auctions

---

#### Por dia

In [14]:
feature = auctions.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
feature = feature.unstack().fillna(0).astype(np.int64)
feature.columns = ["apariciones_en_dia_{}".format(dia) for dia in range(1,4)]


In [15]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Sistema operativo (Android / iOS)

#### Cantidad de so

In [16]:
feature=  auctions.groupby("device_id").agg({"ref_type_id" : "nunique"}).rename(columns={"ref_type_id": "cantidad_so_registrados"})

In [17]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

#### Media so

In [18]:
feature = auctions.groupby("device_id").agg({"ref_type_id" : "mean"}).rename(columns={"ref_type_id": "so_media"})

In [19]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Tiempo medio entre apariciones

In [20]:
gb = auctions.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).mean()
feature = pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_medio_entre_apariciones"})

In [21]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Tiempo máximo entre apariciones

In [22]:
gb = auctions.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).max()
feature = pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_maximo_entre_apariciones"})

In [23]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Desvío entre apariciones

In [24]:
gb = auctions.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).std()
feature=  pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "desvio_entre_apariciones"})

In [25]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Cantidad de apariciones por fuente

In [26]:
feature = auctions.groupby(["device_id", "source_id"])["date"].agg("count").to_frame().unstack()
feature.columns = ["apariciones_en_source_{}".format(x) for x in range(10)]
feature = feature.fillna(0).astype(np.int64)

In [27]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Tiempo desde la última aparición hasta el fin de la ventana

In [28]:
tiempo_final = "2019-04-27"

In [29]:
feature =(np.datetime64(tiempo_final) - auctions.groupby("device_id")\
          .agg({"date" : "max"})["date"]).dt.total_seconds().to_frame()\
        .rename(columns={"date": "tiempo_ultima_aparicion_hasta_fin_ventana"})

In [30]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Tiempo desde t0 hasta horas más frecuentes

#### Funciones para evitar codigo repetido

In [31]:
def obtener_horas_aparicion_maxima(dataframe):
    """Devuelve un dataframe con la pinta {'device_id','hora','date'} donde cada dispositivo
    contiene todas las horas donde su aparición en subastas sea igual a la maxima """
    
    feature = dataframe.groupby(["device_id","hora"]).agg({"date":"count"})
    idx = feature.groupby(["device_id"])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    feature.reset_index(inplace = True)
    return feature

In [32]:
def obtener_tiempo_hasta_hora_minima(feature):
    """Recibe un dataframe feature con horas de aparicion máxima y devuelve el
    tiempo hasta la hora minima para cada dispositivo"""
    segundos_en_hora = 60*60

    feature = feature.reset_index().groupby("device_id").agg({"hora":"min"})
    feature["hora"] = feature["hora"] * segundos_en_hora
    return feature
    

---

### Tiempo desde t0 hasta hora más frecuente de aparicion

#### Frente a empate entre horas me quedaré con la minima (la mas cercana a t0)

In [33]:
auctions["hora"] = auctions["date"].dt.hour


#### Tiempo hasta hora minima

In [34]:
segundos_en_hora = 60*60
    
feature = obtener_horas_aparicion_maxima(auctions)
    
    #Obtengo hora más cerca de t0 = 00 hs
feature = obtener_tiempo_hasta_hora_minima(feature)
feature.rename(columns = {"hora":"tiempo_hora_frec_aparicion_subasta"},inplace = True)
    


In [35]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Tiempo desde t0 hasta la segunda hora más frecuente de aparición

In [36]:
feature = auctions.groupby(["device_id","hora"]).agg({"date":"count"})
feature.reset_index(inplace = True)
    
    
apariciones_maximas = obtener_horas_aparicion_maxima(auctions)
apariciones_maximas.reset_index(inplace = True)

    
    
#Obtengo los indices de las horas minimas de aparicion maxima para cada dispositivo
horas_minimas = apariciones_maximas.groupby("device_id")["hora"].transform(min) == apariciones_maximas["hora"]

apariciones_maximas = apariciones_maximas[horas_minimas]

    
#Me quedo con los datos que no esten entre las horas minimas de aparicion maxima
horas_no_minimas = ~( (feature.isin(apariciones_maximas)))
#Todas las columnas deben ser True
horas_no_minimas = horas_no_minimas.all(1)
    
        
#Me quedo con los datos para completar aquellos dispositivos que solo contienen datos de una sola hora
datos_restantes = feature[ ~(horas_no_minimas)]

    
#Me quedo con los datos que excluyena las horas minimas 
feature = feature[horas_no_minimas]
    
#obtengo las apariciones maximas dentro de los valores restantes 
feature = obtener_horas_aparicion_maxima(feature)
    
#obtengo las horas minimas dentro de los valores restantes( segundas horas minimas)
feature = obtener_tiempo_hasta_hora_minima(feature)
        
#Me quedo con los datos que contienen solo una hora 
apariciones_unicas = ~( (datos_restantes["device_id"].isin(feature.index)))
    
datos_restantes = datos_restantes[apariciones_unicas]
  
datos_restantes = obtener_tiempo_hasta_hora_minima(datos_restantes)
     
feature = feature.append(datos_restantes)
    
feature.rename(columns = {"hora":"tiempo_segunda_hora_frec_aparicion"},inplace = True)


In [37]:
targets_individuales = targets_individuales.merge(feature, how = 'left', right_index = True, left_index = True)

---

### Guardo features

In [ ]:
features_auctions = targets_individuales

In [45]:
features_auctions.to_pickle("features_auctions_competencia")

---

# Features Events

In [5]:
fecha_inicio = "2019-04-24"

In [6]:
feacha_fin = "2019-04-27"

In [7]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [8]:
events = events[events["date"] > fecha_inicio]

In [9]:
events["dayofweek"] = (events["date"].dt.dayofweek).astype("int8")

In [10]:
targets_individuales = targets["ref_hash"].drop_duplicates().to_frame()\
                        .rename(columns = {"ref_hash":"device_id"})\
                        .set_index("device_id")

---

## Creación de features

---

## Función generadora de features

In [11]:
def generar_feature_en_ventanas(ventana, generador_feature, destinos):
                    
        feature = generador_feature(ventana)
        
        
        destinos = destinos.merge(feature, how = 'left', left_index=True, right_index=True)
        
        return destinos 

## Cantidad de eventos dentro de la ventna

In [12]:
def cantidad_eventos(dataframe):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cant_eventos"})

In [13]:
targets_individuales = generar_feature_en_ventanas(events,cantidad_eventos,targets_individuales)

## Promedio de eventos por día

In [14]:
def prom_eventos(dataframe):
    feature = dataframe.groupby(["device_id","dayofweek"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"mean"})
    feature["date"] = feature["date"].astype("int64")
    return feature.rename(columns={"date": "prom_eventos"})

In [15]:
targets_individuales = generar_feature_en_ventanas(events, prom_eventos, targets_individuales)

---

## Cantidad de eventos discrimando periodos

In [16]:
# Creo columnas para luego separar
events['hora'] = events['date'].dt.hour
events['dia'] = events['date'].dt.day
events.rename(columns = {"dayofweek":"dia_de_semana"},inplace = True)

In [17]:
def cantidad_apariciones_por_dia(dataframe):
    dia_final = 3
    
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["eventos_dia_{}".format(dia) for dia in range(1,dia_final+1)]
    return apariciones_por_dia

In [18]:
targets_individuales = generar_feature_en_ventanas(events, cantidad_apariciones_por_dia, targets_individuales)

---

## Cantidad de veces que se realizó el evento más frecuente

In [19]:
def cantidad_maximo_evento(dataframe):
    
    feature = events.groupby(["device_id","event_id"]).agg({"date":"count"})
    feature = feature.groupby("device_id").agg({"date":"max"})
    feature.rename(columns = {"date":"max_cant_eventos"},inplace = True)
    
    return feature
    

In [20]:
targets_individuales = generar_feature_en_ventanas(events, cantidad_maximo_evento, targets_individuales)

## Cuantos eventos comparten la primera posición de evento más frecuente

In [21]:
def cantidad_eventos_maximos(dataframe):
    
    feature = events.groupby(["device_id","event_id"]).agg({"date":"count"})
    idx = feature.groupby(['device_id'])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    feature = feature.groupby("device_id").agg({"date":"count"})
    feature.rename(columns = {"date":"cant_eventos_maximos"},inplace = True)
    
    return feature

In [22]:
targets_individuales = generar_feature_en_ventanas(events, cantidad_eventos_maximos, targets_individuales)

---

## Tiempo en segundos desde 't0' hasta hora más frecuente

In [23]:
events["hora"] = events["date"].dt.hour


In [24]:
def tiempo_hasta_hora_frecuente(dataframe):
    segundos_en_hora = 60*60
    
    feature = dataframe.groupby(["device_id","hora"]).agg({"date":"count"})
    idx = feature.groupby(["device_id"])["date"].transform(max) == feature["date"]
    feature = feature[idx]
    
    feature = feature.reset_index().groupby("device_id").agg({"hora":"min"})
    feature["hora"] = feature["hora"] * segundos_en_hora
    feature.rename(columns = {"hora":"segs_hasta_hora_freq"},inplace = True)
    
    return feature
    
    

In [25]:
targets_individuales = generar_feature_en_ventanas(events,tiempo_hasta_hora_frecuente, targets_individuales)

### Cantidad de eventos con y sin wifi

In [27]:
events["sin_wifi"] = ~(events["wifi"])

In [28]:
def cant_eventos_wifi(dataframe):
    return dataframe.groupby("device_id").agg({"wifi":"sum","sin_wifi":"sum"}).rename(columns={"wifi": "cant_eventos_wifi",
                                                                                              "sin_wifi":"cant_eventos_sin_wifi"})
    

In [29]:
targets_individuales = generar_feature_en_ventanas(events,cant_eventos_wifi, targets_individuales)

### Eventos principales

In [30]:
#Top 10 de eventos con la cantidad de dispositivos registrados
#top_eventos = {"15":"103047","23":"101109","7":"73239","0":"47197","2":"38669","1":"30154","13":"16845","101":"15778","100":"15539","245":"14631"}
eventos_frecuentes = [15,23,7,0,2,1,13,101,100,245]

In [31]:
def cantidad_de_evento_x(dataframe):
    
    return dataframe.groupby("device_id").agg({"es_evento_x":"sum"})\
                                        .rename(columns={"es_evento_x": "cant_eventos_{}".format(EVENTO_ACTUAL)})
    
    
    
    

#### Bucle para generar el feature para un determinado numero de eventos principales

In [32]:
TOP_EVENTOS = 5

In [33]:
for x in range(TOP_EVENTOS):
    EVENTO_ACTUAL = eventos_frecuentes[x]
    events["es_evento_x"] = events["event_id"] == EVENTO_ACTUAL
    
    targets_individuales = generar_feature_en_ventanas(events,cantidad_de_evento_x, targets_individuales)

---

### Distribución de cantidad de eventos por franja horaria

In [34]:
events['hora'] = events['date'].dt.hour


#### Etiquetado de franja

In [37]:
def etiquetar_franja(dataframe,tam_franja):
    
    dataframe["franja"] = 0
    cant_divisiones = 24//tam_franja
    hora_inicio = 0
    hora_fin = tam_franja
    for x in range(1, cant_divisiones + 1):
        dataframe["franja"][(dataframe["hora"] >= hora_inicio) & (dataframe["hora"] < hora_fin)] = x
        hora_inicio += tam_franja  
        hora_fin = hora_inicio + tam_franja


In [38]:
tam_franja = 4
etiquetar_franja(events,tam_franja)

/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [40]:
def cantidad_eventos_por_franja(dataframe):
    cant_divisiones = 24//tam_franja
    apariciones_por_franja = dataframe.groupby(["device_id", "franja"]).agg(({"date" : "count"}))
    apariciones_por_franja = apariciones_por_franja.unstack().fillna(0).astype(np.int64)
    apariciones_por_franja.columns = ["eventos_franja_{}".format(franja) for franja in range(1,cant_divisiones + 1)]
    return apariciones_por_franja
    

In [41]:
targets_individuales = generar_feature_en_ventanas(events,cantidad_eventos_por_franja, targets_individuales)

---

### Guardo features

In [42]:
features_events = targets_individuales

In [43]:
features_events.to_pickle("features_events_competencia")

---

# Features Clicks

In [130]:
clicks = pd.read_pickle("../../../../data/tp2/clicks_tp2_formateado.pkl")

In [131]:
fecha_inicio = "2019-04-24"

In [132]:
fecha_fin = "2019-04-27"

In [133]:
clicks = clicks[clicks["created"] > fecha_inicio]

In [134]:
targets_individuales = targets["ref_hash"].drop_duplicates().to_frame()\
                        .rename(columns = {"ref_hash":"device_id"})\
                        .set_index("device_id")

## Creación de features

---

### Función generadora de features

In [135]:
def generar_feature_en_ventanas(ventana, generador_feature, destinos):
                    
        feature = generador_feature(ventana)
        
        
        destinos = destinos.merge(feature, how = 'left', left_index=True, right_index=True)
        
        return destinos 

### Estadísticas sobre el tiempo hasta clickear

In [136]:
def tiempo_hasta_clickear_media(dataframe):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "mean"}).rename(columns={"timeToClick": "tiempo_hasta_click_mean"})

In [137]:
targets_individuales = generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_media, targets_individuales)

In [138]:
def tiempo_hasta_clickear_max(dataframe):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "max"}).rename(columns={"timeToClick": "tiempo_hasta_click_max"})

In [139]:
targets_individuales = generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_max, targets_individuales)

In [140]:
def tiempo_hasta_clickear_min(dataframe):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "min"}).rename(columns={"timeToClick": "tiempo_hasta_click_min"})

In [141]:
targets_individuales = generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_min, targets_individuales)

In [142]:
def tiempo_hasta_clickear_std(dataframe):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "std"}).rename(columns={"timeToClick": "tiempo_hasta_click_std"}).fillna(0)

In [143]:
targets_individuales = generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_std, targets_individuales)

---

## Cantidad de clicks dentro de la ventana

In [144]:
def cantidad_de_clicks(dataframe):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "count"}).rename(columns={"timeToClick": "cantidad_clicks"})

In [145]:
targets_individuales = generar_feature_en_ventanas(clicks, cantidad_de_clicks, targets_individuales)

---

## Cantidad de clicks discriminando por períodos

In [146]:
# Creo columnas para luego separar
clicks['hora'] = clicks['created'].dt.hour
clicks['dia'] = clicks['created'].dt.day
clicks['dia_de_la_semana'] = clicks['created'].dt.dayofweek

In [147]:
def cantidad_clicks_hora(dataframe):
    apariciones_por_hora = dataframe.groupby(["ref_hash", "hora"]).agg(({"created" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().fillna(0).astype(np.int64)
    apariciones_por_hora.columns = ["clicks_en_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

In [148]:
targets_individuales = generar_feature_en_ventanas(clicks, cantidad_clicks_hora, targets_individuales)

In [149]:
def cantidad_clicks_dia(dataframe):
    dias = dataframe["dia"].drop_duplicates
    apariciones_por_dia = dataframe.groupby(["ref_hash", "dia"]).agg(({"created" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["clicks_en_dia_{}".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [150]:
targets_individuales = generar_feature_en_ventanas(clicks, cantidad_clicks_dia, targets_individuales)

## Tiempo medio entre clicks

In [151]:
def tiempo_medio_entre_clicks(dataframe):
    gb = dataframe.groupby("ref_hash")
    ps = {}
    for t, g in gb:
        ps[t] = g["created"].sub(g["created"].shift()).mean()
    return pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_medio_entre_clicks"})

In [152]:
targets_individuales = generar_feature_en_ventanas(clicks, tiempo_medio_entre_clicks, targets_individuales)

### Guardo features

In [153]:
features_clicks = targets_individuales

In [154]:
features_clicks.to_pickle("features_clicks_competencia")

---

# Features Installs

In [76]:
installs = pd.read_pickle("../../../../data/tp2/installs_tp2_formateado.pkl")

In [77]:
feacha_inicio = "2019-04-24"

In [78]:
fecha_fin = "2019-04-27"

In [79]:
installs = installs[installs["created"] > fecha_inicio]

In [80]:
targets_individuales = targets["ref_hash"].drop_duplicates().to_frame()\
                        .rename(columns = {"ref_hash":"device_id"})\
                        .set_index("device_id")

## Cantidad de instalaciones dentro de la ventana

In [81]:
def cantidad_de_instalaciones(dataframe):
    return dataframe.groupby("ref_hash").agg({"created" : "count"}).rename(columns={"created": "cantidad_instalaciones"})

In [82]:
targets_individuales = generar_feature_en_ventanas(installs, cantidad_de_instalaciones, targets_individuales)

---

## Tiempo desde la última aparición hasta el fin de la ventana

In [86]:
def tiempo_desde_ult_instalacion_hasta_fin_ventana(dataframe):
    return (np.datetime64(fecha_fin) - dataframe.groupby("ref_hash").agg({"created" : "max"})["created"]).dt.total_seconds().to_frame().rename(columns={"created": "tiempo_ultima_instalacion_hasta_fin_ventana"})

In [87]:
targets_individuales = generar_feature_en_ventanas(installs, tiempo_desde_ult_instalacion_hasta_fin_ventana, targets_individuales)

---

## Cantidad de aplicaciones distintas instaladas

In [88]:
def cantidad_aplicaciones_distintas_instaladas(dataframe):
    return dataframe.groupby("ref_hash").agg({"application_id" : "nunique"}).rename(columns={"application_id": "cantidad_aplicaciones_diferentes"})

In [89]:
targets_individuales = generar_feature_en_ventanas(installs, cantidad_aplicaciones_distintas_instaladas, targets_individuales)

---

### Guardado de features

In [90]:
features_clicks = targets_individuales

In [91]:
features_clicks.to_pickle("features_installs_competencia")

---

## Armado de set de features completo

In [5]:
features_auctions = pd.read_pickle("features_auctions_competencia")
features_events = pd.read_pickle("features_events_competencia")
features_clicks = pd.read_pickle("features_clicks_competencia")
features_installs = pd.read_pickle("features_installs_competencia")

In [6]:
features = targets["ref_hash"].drop_duplicates().to_frame()\
                        .rename(columns = {"ref_hash":"device_id"})\
                        .set_index("device_id")

In [7]:
features = features.merge(features_auctions, how = 'left', left_index= True, right_index = True)
features = features.merge(features_events, how = 'left', left_index= True, right_index = True)
features = features.merge(features_clicks, how = 'left', left_index= True, right_index = True)
features = features.merge(features_installs, how = 'left', left_index= True, right_index = True)

In [8]:
features.to_pickle("features_completos_competencia")

### Cuanto es la maxima cantidad de nulos que hay dentro de los datos

In [9]:
datos_no_nulos = features[~features.isna().all(axis = 1)]

In [10]:
datos_no_nulos["cant_nulos"] = datos_no_nulos.isnull().sum(axis = 1)

/home/pelozo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
datos_no_nulos["cant_nulos"].max()

99

# Tratamiento de valores nulos

In [8]:
features_completos = features

In [12]:
features.isna().all(axis = 1).sum()

349

In [12]:
devices_nulos = features[features.isna().all(axis = 1)].index

In [13]:
targets_nulos = devices_nulos.to_frame().drop("device_id",1)

### Obtengo los features recopilados anteriormente para esos valores y saco la media

In [14]:
cantidad_ventanas = 5

In [15]:
features_nulos = pd.DataFrame()

In [16]:
for ventana_nro in range(1,cantidad_ventanas+1):
    
    features_auctions = pd.read_pickle("../../../modelo/features/auctions_train_ventana_{}.pkl".format(ventana_nro))
    features_clicks = pd.read_pickle("../../../modelo/features/clicks_train_ventana_{}.pkl".format(ventana_nro))
    features_events = pd.read_pickle("../../../modelo/features/events_train_ventana_{}.pkl".format(ventana_nro))
    features_installs = pd.read_pickle("../../../modelo/features/installs_train_ventana_{}.pkl".format(ventana_nro))
 

    features = targets_nulos.merge(features_auctions, how = 'left', left_index=True, right_index=True)
    
    features = features.merge(features_events,how = 'left', left_index = True, right_index = True)

    features = features.merge(features_clicks,how = 'left',left_index = True, right_index = True)
    
    features = features.merge(features_installs, how = 'left', left_index = True, right_index = True)
    
    features_nulos = features_nulos.append(features)


In [17]:
len(features_nulos)

1745

In [18]:
features_nulos.shape

(1745, 103)

In [19]:
imputacion = features_nulos.groupby("device_id").median()

In [20]:
imputacion.isna().any(axis = 1).sum()

346

In [21]:
len(imputacion)

349

In [22]:
imputacion.head()

,cantidad_apariciones,apariciones_en_hora_0,apariciones_en_hora_1,apariciones_en_hora_2,apariciones_en_hora_3,apariciones_en_hora_4,apariciones_en_hora_5,apariciones_en_hora_6,apariciones_en_hora_7,apariciones_en_hora_8,...,clicks_en_hora_21,clicks_en_hora_22,clicks_en_hora_23,clicks_en_dia_1,clicks_en_dia_2,clicks_en_dia_3,tiempo_medio_entre_clicks,cantidad_instalaciones,tiempo_ultima_instalacion_hasta_fin_ventana,cantidad_aplicaciones_diferentes
device_id,,,,,,,,,,,,,,,,,,,,,
97694307709005918,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,133617.327,1.0
104412862089502190,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,149277.585,1.0
127181479361225903,9.0,0.0,0.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,58409.113,1.0
162252644031331242,13.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,104015.291,1.0
177763697089045655,1.5,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,141900.939,1.0


In [23]:
features_con_imputacion = features_completos[~features_completos.isna().all(axis = 1)]

In [24]:
features_con_imputacion = features_con_imputacion.append(imputacion)

/home/pelozo/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [25]:
features_con_imputacion.to_pickle("features_con_imputacion.pkl")

In [26]:
features_con_imputacion.shape

(4037, 103)